In [3]:
# %pip install spacy
import spacy
from spacy import displacy
from IPython.core.display import display, HTML


display(HTML("<style>pre { white-space: pre !important; }</style>"))

Note: you may need to restart the kernel to use updated packages.

In [4]:
import matplotlib.pyplot as plt
from collections import OrderedDict
import seaborn as sns

import pandas as pd



import numpy as np

In [5]:
import pyspark

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.ml.stat import Correlation
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType, StringType
import matplotlib.pyplot as plt 
import seaborn as sns
import pyspark.sql.types as T
import pyspark.sql.functions as F

In [7]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .master("local[4]") \
    .config("spark.driver.maxResultSize", "8g") \
    .getOrCreate()

In [9]:
artist_df = spark.read.option("header", "true").csv("../data/artists.csv")
artist_df.printSchema()
artist_df.show(10)

root
 |-- id: string (nullable = true)
 |-- followers: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- name: string (nullable = true)
 |-- popularity: string (nullable = true)

+--------------------+---------+------+--------------------+----------+
|                  id|followers|genres|                name|popularity|
+--------------------+---------+------+--------------------+----------+
|0DheY5irMjBUeLybb...|      0.0|    []|Armid & Amir Zare...|         0|
|0DlhY15l3wsrnlfGi...|      5.0|    []|         ปูนา ภาวิณี|         0|
|0DmRESX2JknGPQyO1...|      0.0|    []|               Sadaa|         0|
|0DmhnbHjm1qw6NCYP...|      0.0|    []|           Tra'gruda|         0|
|0Dn11fWM7vHQ3rinv...|      2.0|    []|Ioannis Panoutsop...|         0|
|0DotfDlYMGqkbzfBh...|      7.0|    []|       Astral Affect|         0|
|0DqP3bOCiC48L8SM9...|      1.0|    []|           Yung Seed|         0|
|0Drs3maQb99iRglyT...|      0.0|    []|               Wi'Ma|         0|
|0DsPeAi1gx

In [10]:
tracks_df = spark.read.option("header", "true").csv("../data/tracks.csv")
tracks_df.show()
tracks_df.printSchema()
tracks_df.count()

+--------------------+--------------------+----------+-----------+--------+-------------------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+
|                  id|                name|popularity|duration_ms|explicit|            artists|          id_artists|release_date|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|
+--------------------+--------------------+----------+-----------+--------+-------------------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+
|35iwgR4jXetI318WE...|               Carve|         6|     126903|       0|            ['Uli']|['45tIt06XoI0Iio4...|  1922-02-22|       0.645| 0.445|  0| -13.338|   1|      0.451|       0.674|           0.744|   0.151|  0.127|104.851|             3|


586672

# Data Cleaning

In [11]:
tracks_df.select("explicit").distinct().show(10)

+---------+
| explicit|
+---------+
|   687600|
|   362760|
|   240200|
|   296467|
|   404000|
|  Melot)"|
|   211160|
|  1191416|
|   185347|
| Hoffmann|
+---------+
only showing top 10 rows



In [12]:
#missing values
tracks_df.select([count(when(isnan(c), c)).alias(c) for c in tracks_df.columns]).show()
artist_df.select([count(when(isnan(c), c)).alias(c) for c in artist_df.columns]).show()

+---+----+----------+-----------+--------+-------+----------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+--------------+
| id|name|popularity|duration_ms|explicit|artists|id_artists|release_date|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|tempo|time_signature|
+---+----+----------+-----------+--------+-------+----------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+--------------+
|  0|   0|         0|          0|       0|      0|         0|           0|           0|     0|  0|       0|   0|          0|           0|               0|       0|      0|    0|             0|
+---+----+----------+-----------+--------+-------+----------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+--------------+

+---+---------+------+----+-------

In [13]:
#null values
tracks_df.select([count(when(col(c).isNull(), c)).alias(c) for c in tracks_df.columns]).show()
artist_df.select([count(when(col(c).isNull(), c)).alias(c) for c in artist_df.columns]).show()

+---+----+----------+-----------+--------+-------+----------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+--------------+
| id|name|popularity|duration_ms|explicit|artists|id_artists|release_date|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|tempo|time_signature|
+---+----+----------+-----------+--------+-------+----------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+--------------+
|  0|  71|         0|          0|       0|      0|        12|          12|          12|    12| 12|      12|  12|         12|          12|              12|      12|     12|   12|            12|
+---+----+----------+-----------+--------+-------+----------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+--------------+

+---+---------+------+----+-------

In [14]:
artist_df = artist_df.withColumn("followers", artist_df.followers.cast(IntegerType())) \
         .withColumn("popularity", artist_df.popularity.cast(IntegerType()))

In [15]:
tracks_df = tracks_df.withColumn("duration_ms", tracks_df.duration_ms.cast(IntegerType())) \
         .withColumn("popularity", tracks_df.popularity.cast(IntegerType())) \
         .withColumn("explicit", tracks_df.explicit.cast(IntegerType())) \
         .withColumn("release_date", tracks_df.release_date.cast(DateType())) \
         .withColumn("danceability", tracks_df.danceability.cast(FloatType())) \
         .withColumn("energy", tracks_df.energy.cast(FloatType())) \
         .withColumn("key", tracks_df.key.cast(IntegerType())) \
         .withColumn("loudness", tracks_df.loudness.cast(FloatType())) \
         .withColumn("mode", tracks_df.mode.cast(IntegerType())) \
         .withColumn("speechiness", tracks_df.speechiness.cast(FloatType())) \
         .withColumn("acousticness", tracks_df.acousticness.cast(FloatType())) \
         .withColumn("instrumentalness", tracks_df.instrumentalness.cast(FloatType())) \
         .withColumn("liveness", tracks_df.liveness.cast(FloatType())) \
         .withColumn("valence", tracks_df.valence.cast(FloatType())) \
         .withColumn("tempo", tracks_df.tempo.cast(FloatType())) \
         .withColumn("time_signature", tracks_df.time_signature.cast(IntegerType()))  

In [16]:
#null values
tracks_df.select([count(when(col(c).isNull(), c)).alias(c) for c in tracks_df.columns]).show()
artist_df.select([count(when(col(c).isNull(), c)).alias(c) for c in artist_df.columns]).show()

+---+----+----------+-----------+--------+-------+----------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+--------------+
| id|name|popularity|duration_ms|explicit|artists|id_artists|release_date|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|tempo|time_signature|
+---+----+----------+-----------+--------+-------+----------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+--------------+
|  0|  71|      1854|        826|     416|      0|        12|        3309|        2286|   859|430|     268| 160|         91|          64|              45|      34|     27|   23|            55|
+---+----+----------+-----------+--------+-------+----------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+--------------+

+---+---------+------+----+-------

In [17]:
tracks_df = tracks_df.filter(col("release_date").isNotNull())
artist_df= artist_df.withColumn('popularity', coalesce(artist_df['popularity'], lit(0))) \
                         .withColumn('followers', coalesce(artist_df['followers'], lit(0)))


In [18]:
#null values
tracks_df.select([count(when(col(c).isNull(), c)).alias(c) for c in tracks_df.columns]).show()
artist_df.select([count(when(col(c).isNull(), c)).alias(c) for c in artist_df.columns]).show()

+---+----+----------+-----------+--------+-------+----------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+--------------+
| id|name|popularity|duration_ms|explicit|artists|id_artists|release_date|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|tempo|time_signature|
+---+----+----------+-----------+--------+-------+----------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+--------------+
|  0|  71|         0|          0|       0|      0|         0|           0|           0|     0|  0|       0|   0|          0|           0|               0|       0|      0|    0|             0|
+---+----+----------+-----------+--------+-------+----------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+--------------+

+---+---------+------+----+-------

In [19]:
tracks_df.count()

583363

In [20]:
artist_df.count()

1104349

In [21]:
import datetime

tracks_df= tracks_df.withColumn('age',F.datediff(F.current_date(), tracks_df.release_date)/365)

# Data Integration

In [22]:
artist_df = artist_df.withColumn(
    "genres",
    split(regexp_replace(col("genres"), r"(^\[)|(\]$)|(')", ""), ", ")
)

In [23]:
tracks_df_wk0= tracks_df.withColumn(
    "id_artists",
    split(regexp_replace(col("id_artists"), r"(^\[)|(\]$)|(')", ""), ", ")
)
tracks_df_wk0

DataFrame[id: string, name: string, popularity: int, duration_ms: int, explicit: int, artists: string, id_artists: array<string>, release_date: date, danceability: float, energy: float, key: int, loudness: float, mode: int, speechiness: float, acousticness: float, instrumentalness: float, liveness: float, valence: float, tempo: float, time_signature: int, age: double]

In [24]:
windowSpec = Window.partitionBy("id_track") 

In [25]:
tracks_df_wk1 = tracks_df_wk0.select(col("id").alias("id_track"), "duration_ms", col("popularity").alias("popularity_track"),"explicit", explode(tracks_df_wk0.id_artists).alias("id_artist"),"release_date","danceability","energy","key","loudness","mode", "speechiness","acousticness","instrumentalness","liveness","valence","tempo","time_signature","age")



tracks_df_wk2 = tracks_df_wk1.join(artist_df, tracks_df_wk1.id_artist==artist_df.id,"left") \
           .withColumn("sum_artist_followers",sum(col("followers")).over(windowSpec)) \
           .withColumn("sum_artist_popularity",sum(col("popularity")).over(windowSpec)) \
           .withColumn("avg_artist_followers",F.avg(col("followers")).over(windowSpec)) \
           .withColumn("avg_artist_popularity",F.avg(col("popularity")).over(windowSpec)) \
           .withColumn("collect_list_genres", collect_list("genres").over(windowSpec)) \
           .withColumn("collect_list_genres", flatten(col("collect_list_genres"))) \
           .withColumn("collect_list_genres", array_distinct("collect_list_genres")) \
           .withColumn("genres", array_remove("collect_list_genres", "")) \
           .drop("collect_list_genres") \
           .select("id_track", "popularity_track",  "duration_ms", "genres", "release_date","danceability","energy","key","loudness","mode", "speechiness","acousticness","instrumentalness","liveness","valence","tempo","time_signature", "sum_artist_followers", "sum_artist_popularity","avg_artist_followers","avg_artist_popularity","age").distinct()



In [26]:
tracks_df_wk2.select("genres").distinct().show(10, truncate=False)

+----------------------------------------------------------------------------------+
|genres                                                                            |
+----------------------------------------------------------------------------------+
|[mariachi, ranchera]                                                              |
|[chanson, french jazz, french pop]                                                |
|[czech folk, czech rock]                                                          |
|[downtempo, new age]                                                              |
|[colombian rock, latin, latin alternative, latin pop, latin rock, rock en espanol]|
|[peruvian rock, pop peruano, pop reggaeton]                                       |
|[adult standards, vocal jazz]                                                     |
|[irish country, irish folk]                                                       |
|[anime]                                                         

In [27]:
tracks_df_wk2.printSchema()

root
 |-- id_track: string (nullable = true)
 |-- popularity_track: integer (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- genres: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- release_date: date (nullable = true)
 |-- danceability: float (nullable = true)
 |-- energy: float (nullable = true)
 |-- key: integer (nullable = true)
 |-- loudness: float (nullable = true)
 |-- mode: integer (nullable = true)
 |-- speechiness: float (nullable = true)
 |-- acousticness: float (nullable = true)
 |-- instrumentalness: float (nullable = true)
 |-- liveness: float (nullable = true)
 |-- valence: float (nullable = true)
 |-- tempo: float (nullable = true)
 |-- time_signature: integer (nullable = true)
 |-- sum_artist_followers: long (nullable = true)
 |-- sum_artist_popularity: long (nullable = true)
 |-- avg_artist_followers: double (nullable = true)
 |-- avg_artist_popularity: double (nullable = true)
 |-- age: double (nullable = true)



In [28]:
tracks_df_wk2.select([count(when(col(c).isNull(), c)).alias(c) for c in tracks_df_wk2.columns]).show()

+--------+----------------+-----------+------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+--------------+--------------------+---------------------+--------------------+---------------------+---+
|id_track|popularity_track|duration_ms|genres|release_date|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|tempo|time_signature|sum_artist_followers|sum_artist_popularity|avg_artist_followers|avg_artist_popularity|age|
+--------+----------------+-----------+------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+--------------+--------------------+---------------------+--------------------+---------------------+---+
|       0|               0|          0|     0|           0|           0|     0|  0|       0|   0|          0|           0|               0|       0|      0|    0|             0|               1103

In [29]:
df = tracks_df_wk2.withColumn('sum_artist_followers', coalesce(tracks_df_wk2['sum_artist_followers'], lit(0))) \
                  .withColumn('sum_artist_popularity', coalesce(tracks_df_wk2['sum_artist_popularity'], lit(0)))\
                  .withColumn('avg_artist_followers', coalesce(tracks_df_wk2['avg_artist_followers'], lit(0))) \
                  .withColumn('avg_artist_popularity', coalesce(tracks_df_wk2['avg_artist_popularity'], lit(0)))\

In [30]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in tracks_df_wk2.columns]).show()

+--------+----------------+-----------+------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+--------------+--------------------+---------------------+--------------------+---------------------+---+
|id_track|popularity_track|duration_ms|genres|release_date|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|tempo|time_signature|sum_artist_followers|sum_artist_popularity|avg_artist_followers|avg_artist_popularity|age|
+--------+----------------+-----------+------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+--------------+--------------------+---------------------+--------------------+---------------------+---+
|       0|               0|          0|     0|           0|           0|     0|  0|       0|   0|          0|           0|               0|       0|      0|    0|             0|                   

# dataset saving

In [31]:
df.write.parquet('../data/cleanedDataset_parquet/')

In [56]:
df_filtered = df.filter(col('popularity_track')>0)

In [61]:
df_filtered.write.mode("overwrite").parquet('../data/cleanedDatasetFiltered_parquet/')

# Correlation

In [37]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler

In [38]:
from pyspark.sql import functions as f

### Normalization

In [40]:
columns_to_scale = ["popularity_track",  "duration_ms", "danceability","energy", "loudness", "speechiness","acousticness","instrumentalness","liveness","valence","tempo","time_signature", "avg_artist_followers", "avg_artist_popularity",  "sum_artist_followers", "sum_artist_popularity","age"]
assemblers = [VectorAssembler(inputCols=[col], outputCol=col + "_vec") for col in columns_to_scale]
scalers = [MinMaxScaler(inputCol=col + "_vec", outputCol=col + "_scaled") for col in columns_to_scale]
pipeline = Pipeline(stages=assemblers + scalers)
scalerModel = pipeline.fit(df)
enriched_df = scalerModel.transform(df)

In [52]:

names = {x + "_scaled": x for x in columns_to_scale}
scaledData = enriched_df.select([f.col(c).alias(names[c]) for c in names.keys()])

In [53]:
scaledData.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+
|    popularity_track|         duration_ms|        danceability|              energy|            loudness|         speechiness|        acousticness|    instrumentalness|            liveness|             valence|               tempo|      time_signature|sum_artist_followers|sum_artist_popularity|                 age|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+
|               [0.0]|[0.3012114547246877]|[0.

In [52]:
scaledData.printSchema()

root
 |-- popularity_track: vector (nullable = true)
 |-- duration_ms: vector (nullable = true)
 |-- danceability: vector (nullable = true)
 |-- energy: vector (nullable = true)
 |-- loudness: vector (nullable = true)
 |-- speechiness: vector (nullable = true)
 |-- acousticness: vector (nullable = true)
 |-- instrumentalness: vector (nullable = true)
 |-- liveness: vector (nullable = true)
 |-- valence: vector (nullable = true)
 |-- tempo: vector (nullable = true)
 |-- time_signature: vector (nullable = true)
 |-- sum_artist_followers: vector (nullable = true)
 |-- sum_artist_popularity: vector (nullable = true)



CORRELATION

In [39]:
from pyspark.ml.stat import Correlation

In [40]:
import pandas as pd

In [56]:
vector_col = "corr_features"
assembler = VectorAssembler(inputCols=scaledData.columns, outputCol=vector_col)
df_vector = assembler.transform(scaledData).select(vector_col)


matrix = Correlation.corr(df_vector, vector_col)
corrmatrix = matrix.collect()[0]["pearson({})".format(vector_col)].values

In [57]:
pd.DataFrame(corrmatrix.reshape(-1, len(scaledData.columns)), columns=scaledData.columns, index=scaledData.columns)

,popularity_track,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,sum_artist_followers,sum_artist_popularity,age
popularity_track,1.000000,0.028081,0.182430,0.298590,0.324094,-0.048209,-0.367301,-0.237153,-0.048200,-0.001050,0.069310,0.085550,0.238490,0.298819,-0.588227
duration_ms,0.028081,1.000000,-0.120047,0.025354,0.000329,-0.126162,-0.065286,0.068597,0.002507,-0.163324,-0.000705,0.037696,0.026750,0.067169,-0.054804
danceability,0.182430,-0.120047,1.000000,0.235541,0.244946,0.198637,-0.237201,-0.225120,-0.106398,0.524472,-0.044804,0.144939,0.035361,-0.019843,-0.221852
energy,0.298590,0.025354,0.235541,1.000000,0.763324,-0.054681,-0.713503,-0.196341,0.125956,0.367742,0.228457,0.188273,0.088351,0.005348,-0.457160
loudness,0.324094,0.000329,0.244946,0.763324,1.000000,-0.169364,-0.516493,-0.330341,0.030301,0.269785,0.187479,0.163391,0.116417,-0.021696,-0.445246
speechiness,-0.048209,-0.126162,0.198637,-0.054681,-0.169364,1.000000,0.070350,-0.102147,0.207660,0.045526,-0.087689,-0.115037,-0.020422,0.053293,0.082604
acousticness,-0.367301,-0.065286,-0.237201,-0.713503,-0.516493,0.070350,1.000000,0.204924,-0.005490,-0.175230,-0.193245,-0.173144,-0.109548,-0.023411,0.524210
instrumentalness,-0.237153,0.068597,-0.225120,-0.196341,-0.330341,-0.102147,0.204924,1.000000,-0.038537,-0.174835,-0.055352,-0.042454,-0.050270,0.005374,0.222705
liveness,-0.048200,0.002507,-0.106398,0.125956,0.030301,0.207660,-0.005490,-0.038537,1.000000,0.000301,-0.014603,-0.023988,0.001988,0.028340,0.019961
valence,-0.001050,-0.163324,0.524472,0.367742,0.269785,0.045526,-0.175230,-0.174835,0.000301,1.000000,0.132874,0.103954,-0.033620,-0.101226,0.031699


# Correlation and Normalization Filtered dataset

In [57]:
columns_to_scale = ["popularity_track",  "duration_ms", "danceability","energy", "loudness", "speechiness","acousticness","instrumentalness","liveness","valence","tempo","time_signature", "avg_artist_followers", "avg_artist_popularity",  "sum_artist_followers", "sum_artist_popularity","age"]
assemblers = [VectorAssembler(inputCols=[col], outputCol=col + "_vec") for col in columns_to_scale]
scalers = [MinMaxScaler(inputCol=col + "_vec", outputCol=col + "_scaled") for col in columns_to_scale]
pipeline = Pipeline(stages=assemblers + scalers)
scalerModel = pipeline.fit(df_filtered)
enriched_df = scalerModel.transform(df_filtered)


names = {x + "_scaled": x for x in columns_to_scale}
scaledData = enriched_df.select([f.col(c).alias(names[c]) for c in names.keys()])



In [58]:
scaledData.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+---------------------+--------------------+
|    popularity_track|         duration_ms|        danceability|              energy|            loudness|         speechiness|        acousticness|    instrumentalness|            liveness|             valence|               tempo|      time_signature|avg_artist_followers|avg_artist_popularity|sum_artist_followers|sum_artist_popularity|                 age|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----

In [59]:
vector_col = "corr_features"
assembler = VectorAssembler(inputCols=scaledData.columns, outputCol=vector_col)
df_vector = assembler.transform(scaledData).select(vector_col)


matrix = Correlation.corr(df_vector, vector_col)
corrmatrix = matrix.collect()[0]["pearson({})".format(vector_col)].values

In [60]:
pd.DataFrame(corrmatrix.reshape(-1, len(scaledData.columns)), columns=scaledData.columns, index=scaledData.columns)

,popularity_track,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,avg_artist_followers,avg_artist_popularity,sum_artist_followers,sum_artist_popularity,age
popularity_track,1.000000,0.030618,0.180220,0.245412,0.300174,-0.021429,-0.303001,-0.163813,-0.054575,-0.017061,0.050142,0.073711,0.249974,0.500678,0.252504,0.278750,-0.549446
duration_ms,0.030618,1.000000,-0.122167,0.018076,-0.001907,-0.144001,-0.057926,0.066776,0.000865,-0.155149,0.000517,0.037997,0.020008,-0.008406,0.028345,0.057274,-0.036424
danceability,0.180220,-0.122167,1.000000,0.226867,0.236774,0.190077,-0.229768,-0.226152,-0.107738,0.519090,-0.065088,0.144206,0.013611,0.025342,0.031171,-0.002662,-0.235794
energy,0.245412,0.018076,0.226867,1.000000,0.762154,-0.033027,-0.699163,-0.168135,0.129906,0.381022,0.216111,0.186311,0.082188,0.103905,0.080422,-0.010356,-0.404928
loudness,0.300174,-0.001907,0.236774,0.762154,1.000000,-0.141987,-0.516580,-0.326298,0.028419,0.265362,0.174908,0.157412,0.114813,0.111037,0.114397,-0.024614,-0.440851
speechiness,-0.021429,-0.144001,0.190077,-0.033027,-0.141987,1.000000,0.067308,-0.098506,0.223374,0.038749,-0.088402,-0.111807,-0.025037,0.106939,-0.015797,0.090620,0.036688
acousticness,-0.303001,-0.057926,-0.229768,-0.699163,-0.516580,0.067308,1.000000,0.159966,-0.006249,-0.192182,-0.183288,-0.174050,-0.105024,-0.130524,-0.099840,0.003741,0.445165
instrumentalness,-0.163813,0.066776,-0.226152,-0.168135,-0.326298,-0.098506,0.159966,1.000000,-0.033240,-0.180102,-0.043448,-0.039669,-0.046067,-0.078987,-0.041568,0.035427,0.163829
liveness,-0.054575,0.000865,-0.107738,0.129906,0.028419,0.223374,-0.006249,-0.033240,1.000000,-0.001186,-0.014902,-0.024777,0.010306,0.062564,0.004835,0.032671,0.024484
valence,-0.017061,-0.155149,0.519090,0.381022,0.265362,0.038749,-0.192182,-0.180102,-0.001186,1.000000,0.125599,0.104994,-0.040109,-0.054388,-0.037129,-0.089754,0.035224
